### The concrete delivery problem
    J. Kinable , T. Wauters, G. Vanden Berghe, 2014
- Constraints:
    - Concrete is produced at several homogeneous production sites located some distance away from the customers. 
    - Trucks start at a central depot, travel to a production plant, unload their cargo at one of the construction sites.
- Objective:  maximize the number of satisfied customers, weighted by their demand
- Instance:
    - To facilitate a future comparison of CDP implementations, large number of problem instances are published.
- Solution:
    - Integer Programming models
        - first ILP: combination of CVRPTW with Split Deliveries, and the Parallel Machine Scheduling Problem (PMSP) with Time Windows and Maximum Time Lags
            - A truck can only be used once for every customer, and whenever it is used, its delivery must be succeeded by another delivery
        - second ILP: allows a single vehicle to visit customers more than once by representing each customer by a of customer nodes
        - Constraint programming model.
    - Heuristic
        - Steepest Descent and best fit
        - Fix-and-optimize heuristic


### First ILP
- The concrete trucks start their trip at a central (source) depot and travel between production sites and customers.
- At the end of the day, the trucks return to an end (sink) depot (which may not be the same as the starting depot).
- directed, weighted graph G(V,A) $V = \{0\} \cup C \cup \{n+1\} $:
    -  0: source
    - n+1: sink 
    
- Arc costs:
    - $c_{0,i} =   \min_{p \in P} t_{0,p} + t_{p,i}, \forall i \in C.$
    - $c_{i,j} = \min_{p \in P} t_{i,p} + t_{p,j}, \forall i,j \in C.$
    - $c_{i,n+1} =  t_{i,n+1}$
    - $c_{0,n+1} = 0.$
- Parameters:
    - $p_k$: Time required to empty truck $k \in K$
    - $\gamma$: Maximum time lag between consecutive deliveries.
    
- Variables:
    - $x_{ijk}$ is a binary variable indicating whether vehicle k travels from i to j
    - $C^{i}_k$ record the time that vehicle k finishes its delivery to customer i.
    - $y_i$ denotes whether customer $i$ is serviced.
    - $z^{i}_{kl}$ is equal to one if truck $k$ delivers its payload immediately before truck $l$ to customer $i$, zero otherwise. 
    - $k_0$ represents a dummy truck
    - $K_0 = K \cup \{k_0\}$
    
- Model
    - $max \sum_{i \in C} q_iy_i$
    - Ensures that sufficient concrete is delivered at construction site
        - $\sum_{k \in K} \sum_{j \in \delta^{+}(i) }q_kx_{ijk} \geq q_iy_i  ,i \in C$
    - Determine the shape of a feasible tour: a tour starts at the source depot, visits a number of pickup and delivery points and finally returns to the sink depot.
        - $ \sum_{j \in \delta^{+}(0)}x_{0jk} = \sum_{i \in \delta^{-}(n+1)}x_{i,n+1,k} = 1 ,k \in K$
        - $ \sum_{j \in \delta^{+}(i)}x_{ijk} = \sum_{j \in \delta^{-}(i)}x_{j,i,k}$, $c \in C, k \in K$
        - The dummy truck must be scheduled
            - $ \sum_{l\in K_0} z^{i}_{k_0l} = 1$, $i \in V$.
        - Flow preservation constraints, each delivery has exactly one successor and one predecessor
            - $ \sum_{l\in K_0 \backslash \{k\}} z^{i}_{kl} = \sum_{l\in K_0 \backslash \{k\}} z^{i}_{lk}$, $k \neq l$
            
    - Between two consecutive visits, starting, processing and travel times have to be taken into account
        - $ C^{i}_k + t_{ij} - M(1-x_{ijk}) \leq C^{j}_k- p_k $, $i,j \in V, k\in K$
        -  Enforce that deliveries do not overlap in time:
            - $  C^{i}_k - M(1-z^{i}_{kl}) \leq C^{i}_l - p_l $ 
        -  Enforce a maximum time lag between consecutive deliveries
            - $  C^{i}_l - p_l \leq C^{i}_k + \gamma + M(1-z^{i}_{kl}) $ 
    - $a_i + p_k \leq C^{i}_k \leq b_i$, $i \in V$

    - $ \sum_{l \in K_0 \backslash \{k\}}z^{i}_{kl} = \sum_{j \in N}x_{ijk} $, $\forall k \in K, i\ in C$
    
    - $x_{ijk} \in \{0,1\}$ $C^{i}_k \in Z$, $y_i \in \{0,1\}$

### Second ILP
- The concrete trucks start their trip at a central (source) depot and travel between production sites and customers.
- At the end of the day, the trucks return to an end (sink) depot (which may not be the same as the starting depot).
- $ \forall$ customer $i \in C$ , $C^i = \{1,...,n(i)\} $, $n(i)= \lceil {\frac{q_i}{\min_{k \in K}\{q_k\}}} \rceil$
- Time window $[a_u, b_u ] = [a_i,b_i]$ $i \in C, u \in C^i$
- $D = \cup_{i\in C}C^i $
- directed, weighted graph G(V,A) $V = \{0\} \cup D \cup \{n+1\} $:  0: source, n+1: sink 
- delivery node $c^i_h$  has a directed edge to a delivery node $c^i_j$ if $ h < j, i \in C, h,j \in C_i$.
- directed edge from $c^i_u$ to $c^j_v, i \neq j$, except if $c^j_v$ needs to be scheduled earlier than $c^i_u$
- Arc costs:
    - $c_{0,c^i_j} = \min_{p \in P} t_{0,p} + t_{p,i}, \forall c^i_j \in D.$
    - $c_{c^i{u},c^j{v}} = \min_{p \in P} t_{i,p} + t_{p,j}, \forall c^i{u},c^j{v} \in D, c^i{u} \neq c^j{v}$
    - $c_{c^i{j},n+1} =  t_{i,n+1}$
    - $c_{0,n+1} = 0.$



- Parameters:
    - $p_k$: Time required to empty truck $k \in K$
    - $\gamma$: Maximum time lag between consecutive deliveries.
    
- Variables:
    - $x_{ijk}$ is a binary variable indicating whether vehicle k travels from i to j
    - $c^{i}$ records the time that delivery $i \in D$ is completed.
    - $y_i$ denotes whether customer $i$ is serviced.
    
- Model
    - $max \sum_{i \in C} q_iy_i$
    - Determine the shape of a feasible tour: a tour starts at the source depot, visits a number of pickup and delivery points and finally returns to the sink depot.
        - $ \sum_{j \in \delta^{+}(0)}x_{0jk} = \sum_{i \in \delta^{-}(n+1)}x_{i,n+1,k} = 1 ,k \in K$
        - $ \sum_{j \in \delta^{+}(i)}x_{ijk} = \sum_{j \in \delta^{-}(i)}x_{j,i,k}$, $c \in C, k \in K$
    - $S(i,\alpha)$ = $\sum_{k \in K} \sum_{j \in \delta^{+}(i)} \alpha x_{ijk} $, $i \in D$
        - Number of times a delivery can be made
            - $S(i,1) \leq 1$, $\forall i \in D$    
        - Ordering of deliveries
            - $S(j+1,1) \leq S(j,1)$ ,$ \forall i \in C, j \in \{1,...,n(i)-1\}$
        - Sum of capacities of the vehicles performing the deliveries for customer $i \in C$ should cover the customer's demand
            - $\sum_{j \in C^i} S(j,q_k) \geq q_i y_i$, $i \in C$
    - $  c^i - M(1- x_{ijk}) \leq c^j - p_k - c_{ij} $, $i \neq 0$
    - $  c^i - M(1- x_{ijk}) \leq c^j - c_{ij} $, $\forall (0,j) \in A$
    - $  c^i - S(i,p_k) \geq a_i $
    -  Enforce a maximum time lag between consecutive deliveries
        - $  c^{j+1} - S(j+1,p_k) - c^j \leq \gamma $, $i \in C, $j \in \{1,...,n(i)-1\}$
    -  Enforce that deliveries do not overlap in time:
        - $  c^{j+1} \geq c^j + S(j,p_k) , $i \in C, $j \in \{1,...,n(i)-1\}$
   
    - $a_i + p_k \leq C^{i}_k \leq b_i$, $i \in V$

    
    - $x_{ijk} \in \{0,1\}$ $C^{i}_k \in Z$, $y_i \in \{0,1\}$